In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install faker
!pip install nest-asyncio
!apt-get update # to update ubuntu to correctly run apt install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [78.0 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updat

In [ ]:
!pip install playwright
!playwright install


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 35.0 MB/s eta 0:00:00
145.2 Mb [] 0% 0.0s145.2 Mb [] 0% 3.2s145.2 Mb [] 1% 1.8s145.2 Mb [] 2% 1.9s145.2 Mb [] 3% 1.8s145.2 Mb [] 4% 1.8s145.2 Mb [] 5% 1.7s145.2 Mb [] 6% 1.5s145.2 Mb [] 8% 1.5s145.2 Mb [] 9% 1.4s145.2 Mb [] 10% 1.4s145.2 Mb [] 12% 1.3s145.2 Mb [] 13% 1.2s145.2 Mb [] 15% 1.2s145.2 Mb [] 17% 1.1s145.2 Mb [] 17% 1.2s145.2 Mb [] 19% 1.1s145.2 Mb [] 20% 1.1s145.2 Mb [] 22% 1.1s145.2 Mb [] 23% 1.1s145.2 Mb [] 25% 1.0s145.2 Mb [] 27% 1.0s145.2 Mb [] 28% 0.9s145.2 Mb [] 29% 0.9s145.2 Mb [] 31% 0.9s145.2 Mb [] 32% 0.9s145.2 Mb [] 33% 0.9s145.2 Mb [] 34% 0.8s145.2 Mb [] 36% 0.8s145.2 Mb [] 37% 0.8s145.2 Mb [] 38% 0.8s145.2 Mb [] 41% 0.7s145.2 Mb [] 43% 0.7s145.2 Mb [] 44% 0.7s145.2 Mb [] 47% 0.6s145.2 Mb [] 48% 0.6s145.2 Mb [] 50% 0.6s145.2 Mb [] 52% 0.6s145.2 Mb [] 54% 0.5s145.2 Mb [] 56% 0.5s145.2 Mb [] 58% 0.5s145.2 Mb [

In [ ]:
import threading
import asyncio
from concurrent.futures import ThreadPoolExecutor
from faker import Faker
from playwright.async_api import async_playwright
import nest_asyncio

nest_asyncio.apply()
fake = Faker('en_IN')
MUTEX = threading.Lock()

def sync_print(text):
    with MUTEX:
        print(text)

async def start(name, user, wait_time, meetingcode, passcode):
    sync_print(f"{name} started!")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=['--use-fake-device-for-media-stream', '--use-fake-ui-for-media-stream'])
        context = await browser.new_context(permissions=['microphone'])
        page = await context.new_page()
        await page.goto(f'https://zoom.us/wc/join/{meetingcode}', timeout=200000)

        try:
            await page.click('//button[@id="onetrust-accept-btn-handler"]', timeout=5000)
        except:
            pass
        try:
            await page.click('//button[@id="wc_agree1"]', timeout=50000)
        except:
            pass

        await page.wait_for_selector('input[type="text"]', timeout=200000)
        await page.fill('input[type="text"]', user)
        await page.fill('input[type="password"]', passcode)
        join_button = await page.wait_for_selector('button.preview-join-button')
        await join_button.click()

        try:
            # Increase timeout if still mic missing on some users
            query = '//button[text()="Join Audio by Computer"]'
            mic_button_locator = await page.wait_for_selector(query, timeout=200000)
            await mic_button_locator.wait_for_element_state('stable', timeout=200000)
            await mic_button_locator.evaluate_handle('node => node.click()')
            sync_print(f"{name} mic aayenge.")

        except Exception as e:
            print(e)
            sync_print(f"{name} mic nhi aayenge.")

        sync_print(f"{name} sleep for {wait_time} seconds ...")
        await asyncio.sleep(wait_time)
        sync_print(f"{name} ended!")

        await browser.close()

async def main():
    number = int(input("Enter number of Users: "))
    meetingcode = input("Enter meeting code (No Space): ")
    passcode = input("Enter Password (No Space): ")

    sec = 60
    wait_time = sec * 60

    with ThreadPoolExecutor(max_workers=number) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for i in range(number):
            try:
                user = fake.name()
            except IndexError:
                break
            task = loop.create_task(start(f'[Thread{i}]', user, wait_time, meetingcode, passcode))
            tasks.append(task)
        await asyncio.gather(*tasks)

if __name__ == '__main__':
    asyncio.run(main())


Enter number of Users: 10
Enter meeting code (No Space): 84374433260
Enter Password (No Space): alprose
[Thread0] started!
[Thread1] started!
[Thread2] started!
[Thread3] started!
[Thread4] started!
[Thread5] started!
[Thread6] started!
[Thread7] started!
[Thread8] started!
[Thread9] started!
